<a href="https://colab.research.google.com/github/MyNameIsEldar/Movie_Recommender/blob/main/rec_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🎥Movie Recommender
Recommendation System using deep learning

##✔️Introduction

Required to build embeddings for each user and movie using **neural networks methods**. These embeddings will be used to find more relevant film for user.

Solvig this task I will use the DFM. Deep Factorization Machines (DFM) - is a multi-layer neural network to model the user and item latent factors.

DMF can be trained using backpropagation and can handle missing data.

For this task I will use the **full MovieLens 20M dataset**.


##📑Imports

In [5]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import mean_squared_error, roc_auc_score